In [ ]:
!pip install -r ../requirements.txt

In [1]:
import os
import sys
import pandas as pd
from bs4 import BeautifulSoup   
import requests
import re



In [2]:
sys.path.append('..')
from GetData import setdata

In [3]:
url = "https://tonton.amaneku.com/sp/list.php?id=20230612011710_zmxBic"
setdata1 = setdata.SetData(url)
all_data = setdata1.data_user_frame["全員"].replace("", pd.NA)
all_data=all_data.set_index(["日付", "時間帯"])
all_data=all_data.dropna(how='all', axis=0)

In [4]:
all_data

まつながかのん 川村花奏 ひなた しょうよう  ゆ ねもとのかのん あしざわななみ 長田真弥 さじゅ   
日付           時間帯                                                              
2023/6/17(土) 11:00～11:30       ●    ●   ●     ●  ●       ×       ▲    ×   ●  \
             11:30～12:00       ●    ●   ●     ●  ●       ×       ▲    ×   ●   
             12:00～12:30       ●    ●   ●     ●  ●       ×       ▲    ×   ●   
             12:30～13:00       ●    ●   ●     ●  ●       ×       ▲    ×   ●   
             13:00～13:30       ●    ●   ●     ●  ●       ×       ▲    ●   ●   
...                          ...  ...  ..   ... ..     ...     ...  ...  ..   
2023/6/23(金) 18:30～19:00       ●    ●   ×     ●  ×       ×       ●    ●   ●   
             19:00～19:30       ●    ●   ×     ●  ×       ×       ●    ●   ●   
             19:30～20:00       ●    ●   ×     ●  ×       ×       ●    ●   ●   
             20:00～20:30       ●    ●   ×     ●  ×       ×       ●    ●   ●   
             20:30～21:00       ●    ●   ×     ●  ×       ×       ●    ●   ●   

                         アタラシタクマ はるひ 寺嶋みち香 能澤楓 藤川光浩 吉岡秀人 永見  
日付           時間帯                                             
2023/6/17(土) 11:00～11:30       ●   ●     ●   ×    ×    ×  ●  
             11:30～12:00       ●   ●     ●   ×    ×    ×  ●  
             12:00～12:30       ●   ●     ●   ×    ×    ×  ●  
             12:30～13:00       ●   ●     ●   ×    ×    ×  ●  
             13:00～13:30       ●   ●     ●   ×    ×    ×  ●  
...                          ...  ..   ...  ..  ...  ... ..  
2023/6/23(金) 18:30～19:00       ×   ●     ●   ×    ×    ●  ●  
             19:00～19:30       ×   ●     ●   ×    ×    ●  ●  
             19:30～20:00       ×   ●     ●   ×    ×    ●  ●  
             20:00～20:30       ×   ●     ●   ×    ×    ●  ●  
             20:30～21:00       ×   ●     ●   ×    ×    ●  ●  

[64 rows x 16 columns]

In [5]:
# ユーザー集合
user_list = list(all_data.columns)
# スケジューリング対象時間枠の集合("日付", "時間帯")
time_list = list(all_data.index)
# 役職グループ辞書(input)
role_dict = {}
role_dict["演出"] = user_list[0]
role_dict["役者"] = user_list
# 章立てグループ辞書(input)
chap_dict={}
chap_dict["第1章"] = user_list[1:]
chap_dict["第2章"] = user_list[3:5]

In [6]:
# ユーザー集合
user_list = list(all_data.columns)
# スケジューリング対象時間枠の集合("日付", "時間帯")
time_list = list(all_data.index)

In [7]:
from pulp import LpProblem , LpMinimize , LpVariable, lpSum , LpStatus , PULP_CBC_CMD 
from ortoolpy import addvar #LpVariableだけでも可能だけど変数をさっと追加したいときはaddvarが便利
from datetime import datetime, timedelta
import plotly.express as px
import pandas as pd

In [11]:
# PuLPをインポートする
import pulp

# 問題を作る
prob = pulp.LpProblem("PracticeTimeOptimization", pulp.LpMaximize)

# x_ijは団員iが時間帯jに稽古に参加するかどうかを表す二値変数
x = pulp.LpVariable.dicts("x", [(i,j) for i in user_list for j in time_list], cat="Binary")

# 目的関数を設定する
prob += pulp.lpSum([x[(i,j)] for i in user_list for j in time_list])

In [14]:
# 割当禁止制約
def forbid_const():
    global prob
    for user in user_list:
        for time in time_list:
            if all_data[user][time] == "×":
                prob += x[user, time] == 0
forbid_const()

In [20]:
#求解
status = prob.solve()
print('Status:', pulp.LpStatus[status])

#計算結果の表示
import pandas as pd
result = pd.DataFrame([[x[user,time].value() for time in time_list] for user in user_list], index=user_list, columns=time_list)

Status: Optimal


In [22]:
result

,"(2023/6/17(土), 11:00～11:30)","(2023/6/17(土), 11:30～12:00)","(2023/6/17(土), 12:00～12:30)","(2023/6/17(土), 12:30～13:00)","(2023/6/17(土), 13:00～13:30)","(2023/6/17(土), 13:30～14:00)","(2023/6/17(土), 14:00～14:30)","(2023/6/17(土), 14:30～15:00)","(2023/6/17(土), 15:00～15:30)","(2023/6/17(土), 15:30～16:00)",...,"(2023/6/22(木), 19:00～19:30)","(2023/6/22(木), 19:30～20:00)","(2023/6/22(木), 20:00～20:30)","(2023/6/22(木), 20:30～21:00)","(2023/6/23(金), 18:00～18:30)","(2023/6/23(金), 18:30～19:00)","(2023/6/23(金), 19:00～19:30)","(2023/6/23(金), 19:30～20:00)","(2023/6/23(金), 20:00～20:30)","(2023/6/23(金), 20:30～21:00)"
まつながかのん,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
川村花奏,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
ひなた,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
しょうよう,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
ゆ,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
ねもとのかのん,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
あしざわななみ,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
長田真弥,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
さじゅ,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
アタラシタクマ,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
